In [1]:


# if __name__ == '__main__':
#     import argparse
    
#     parser = argparse.ArgumentParser(description='')
#     parser.add_argument('--test_dataset_path', dest='test_dataset_path')
#     parser.add_argument('--qas_path', dest='qas_path')
#     parameter_args = parser.parse_args()

#     test_dataset_path = parameter_args.test_dataset_path
#     QAs_path = parameter_args.qas_path

資料集 建立

In [2]:
test_dataset_path = '../data/test_dataset.json'
# test_dataset_path = '../data/evaluation_netizen_dataset.json'
# test_dataset_path = '../data/test_retrieve_dataset.json'
QAs_path = '../data/General_QAs.json'
config_path = '../config/evaluation_config.json'

import sys
from pathlib import Path
sys.path.append('..')

import os
import json
qa_index = 1
only_for_classification = False

def get_time_text():
    
    import time
    time_stamp = time.time() 
    struct_time = time.localtime(time_stamp)
    log_name = time.strftime("%Y-%m-%d %H-%M-%S", struct_time)

    year = struct_time.tm_year
    month = struct_time.tm_mon
    day = struct_time.tm_mday

    folder_name = f'{year}_{month}_{day}'
    
    return folder_name, log_name

def make_dataset(only_for_classification):
    
    _, build_dataset_time = get_time_text()
    time_number = build_dataset_time.replace('-','').replace(' ','')

    dataset = {}
    dataset['name'] = 'test_dataset'
    dataset['build_dataset_time'] = build_dataset_time
    dataset['build_dataset_time_number'] = time_number
    dataset['only_for_classification'] = only_for_classification
    dataset['data_list'] = []
    dataset['token'] = 0
    
    score_data = {}
    score_data['Classifier'] = 0
    score_data['General'] = 0
    score_data['SQL'] = 0
    score_data['Retrieve'] = 0
    
    dataset['score_data'] = score_data
    dataset['Classifier Accuracy'] = 0
    
    return dataset
    
test_set = make_dataset(only_for_classification)

every_class_test_maximum = 100

from langchain_openai import ChatOpenAI
from langchain.schema import StrOutputParser
from langchain.prompts import ChatPromptTemplate
import os
import re
from chains import make_chain, make_memory, Chain_manager

with open(config_path, 'r') as f: config = json.load(f)
os.environ['OPENAI_API_KEY'] = config['OpenAI_api_key']

chain_manager = Chain_manager(config)
question_splitter, _ = chain_manager.make_chain('Question_Splitter')


一般資料 建立

In [3]:
with open(QAs_path, 'r') as f:
    QAs = json.load(f)

for qa in QAs[:every_class_test_maximum]:
    
    data = {}
    data['qa_index'] = str(qa_index)
    qa_index += 1
    data['class'] = '1'
    data['question'] = qa['question']
    data['only_for_classification'] = only_for_classification
    
    test_set['data_list'].append(data)

查詢資料 建立

In [4]:
questions = [
    ('Intel Processor 300多少錢呀？', 'int', '2990'),
    ('詢價Intel i3-14100F價格？', 'int', '3990'),
    ('查Intel i3-14100價格？', 'int', '4800'),
    ('請幫我查Intel i5-14400F多少錢呀？', 'int', '6800'),
    ('幫查Intel i5-14600K價格？', 'int', '10150'),
    ('想查價Intel i7-14700K？', 'int', '13800'),
    ('查Intel i5-14600KF價格？', 'int', '9600'),
    ('想查Intel i7-14700價格是多少？', 'int', '12900'),
    ('想查Inteli5-14600KF價格？', 'int', '9600'),
    ('想查Intel i5-14400價格？', 'int', '7600'),
    
    # 以下為GPU 
     ('詢價GT710-SL-2GD3-BRK-EVO？', 'int', '1690'),
     ('請查詢華碩GT710-SL-2GD3-BRK-EVO 的價錢？', 'int', '1690'),
     ('詢問GT710 1GD3H LP 的價錢？', 'int', '1450'),
     ('查 微星 GT1030 2GD4 LP OC	 的價錢？', 'int', '2850'),
     ('查詢有一台微星GT1030GPU價錢為3290是哪一台？', 'str', '微星 GT1030 AERO ITX 2GD4 OCV1'),
     ('查有一台品牌是Nvidia，價格為6890元的gpu產品名稱', 'str', 'RTX3050 AERO ITX 8G OCV1'),
     ('想知道微星 RTX3050 AERO ITX 8G OCV1的價錢？', 'int', '6890'),
     ('華碩 DUAL-RTX3060-O12G-V2多少錢？', 'int', '8990'),
     ('想知道微星 RTX3060 VENTUS 2X 12G OC多少錢？', 'int', '8990'),
     ('查詢技嘉 RTX3050 EAGLE OC 6G的價錢？', 'int', '5990'),


    #ram
     ('詢價UMAX 單條8GB DDR5-5600/CL46多少錢？', 'int', '749'),
     ('請查詢UMAX 單條16GB DDR5-5600/CL46的價錢？', 'int', '1299'),
     ('詢問UMAX 單條32GB DDR5-5600/CL46的價錢？', 'int', '2599	'),
     ('請查詢產品名稱為[金士頓 單條16GB DDR5-5600 FURY Beast]的價錢？', 'int', '1580'),
     ('想要查詢產品名稱為，[金士頓 單條16GB DDR5-5600 FURY Beast RGB]的價錢？', 'int', '1750'),
     ('查詢一個品牌為美光Micron，價錢為1399的產品全名？', 'str', '美光Micron Crucial 單條16GB DDR5 5600/CL46'),
     ('想查名稱為[威剛 64GB(雙通32GB*2) DDR5 5600 XPG Lancer RGB/CL36 黑的價格？', 'int', '6199'),   #改這筆
     ('想查價格，其產品名字為[威剛 32GB(雙通16GB*2) DDR5 6000 XPG Lancer/CL30 黑]？', 'int', '3499'), 
     ('想查產品名為[美光Micron Crucial 單條32GB DDR5 5600/CL46]多少錢？，DDR5 5600中間不要加入-的符號', 'int', '2799') , 
     ('想查[美光Micron Crucial 64GB(雙通32GB*2) D5 5600/CL46]的價格？', 'int', '5599') , 
    
    #hdd
     ('詢價UMAX S330 240GB價格？', 'int', '490'),
     ('請查詢UMAX S330 480GB的價錢？', 'int', '880'),
     ('詢問威剛 Ultimate SU650 120G的價錢？', 'int', '350'),
     ('請查威剛 Ultimate SU650 240G的價錢？', 'int', '599'),
     ('WD Blue SA510 250GB的價錢？', 'int', '1100'),
     ('想查WD Blue SA510 500GB價格？', 'int', '1500'),
     ('想查一個hdd的價格，它的產品名稱為美光 Crucial BX500 500G？', 'int', '949'), 
     ('想查[十銓 T-Force Vulcan Z]硬碟的價格？', 'int', '699') ,
     ('查詢有一個品牌叫做[美光Micron Crucial BX]的硬碟，價錢為630元的產品名稱？', 'str', '美光 Crucial BX500 240G'),
     ('查有一台品牌為[三星 Samsung]的硬碟，價格為1529元的產品名稱', 'str', '三星 Samsung 870 EVO 500G'),

    
    
    #cool
     ('想從table叫做cool裡面拉一個資料，資料的名稱為[利民 HR-09 2280 SSD 固態硬碟散熱器/6 mm熱導管/電鍍鰭片/單雙面皆適用]，它的價格？', 'int', '550'),
     ('請查詢[快睿 Frostbit M.2 散熱片/2280/雙導管/可調式散熱鰭片]的價錢？', 'int', '790'),
     ('請從名字為cool的table裡面找資料，[be quiet! PURE ROCK 2 BLACK 4導管/高度15.5cm/TDP:150W【WXHZ】]的價錢？', 'int', '1290'),
     ('查詢品牌為利民$350的散熱器，它的產品名稱？', 'str', '利民 M.2 2280 SSD 固態硬碟散熱片'),
     ('想從cool的table中,查詢一台名字為[利民 LGA17XX-SS2 扣具 /支援AS AK TA PA120]價錢？', 'int', '60'), 
     ('詢價一款散熱器，他的名稱為利民 Phantom Spirit 120 EVO，它的價格？', 'int', '1890'),
     ('利民 M.2 2280 SSD 固態硬碟散熱片/鋁合金/單雙面皆適用】價錢？', 'int', '350'),
     ('想知道從cool的table中，找尋[利民 HR-09 2280 SSD 固態硬碟散熱器/6 mm熱導管/電鍍鰭片/單雙面皆適用]的價格？', 'int', '550'),
     ('詢價[利民 M.2 2280 PRO SSD 固態硬碟散熱片/鋁合金+8 mm純銅導管/單雙面皆適用]的價格？', 'int', '400'),
     ('詢價一款在table中，產品名稱為[快睿 Frostbit M.2 散熱片/2280/雙導管/可調式散熱鰭片]的價錢？', 'int', '790'),
    
]

for question in questions[:every_class_test_maximum]:
    
    data = {}
    data['qa_index'] = str(qa_index)
    qa_index += 1
    data['class'] = '2'
    data['question'] = question[0]
    data['answer'] = question[2]
    data['answer_type'] = question[1]
    data['only_for_classification'] = only_for_classification
    test_set['data_list'].append(data)

推薦 資料建立

In [5]:
questions = [
    '請問3萬內能組一台什麼樣的主機，主要是玩LOL和看Netflix用？',
    '有推薦的組合嗎？預算大概在50K，要能處理攝影後製和輕遊戲，如：軒轅劍7。',
    '碩士生研究用的電腦，預算約70K，需要跑深度學習模型，有沒有建議配置？',
    '預算大約20K，尋求一台能寫報告和上網的簡易電腦，要求瀏覽器多開流暢。',
    '想請問80K左右，能組出來什麼級別的遊戲主機？想玩些新遊戲，畫質要好。',
    '工作需要，我要一台能跑Solidworks和AutoCAD的工作站，預算大概60K，有建議嗎？',
    '有沒有推薦的遊戲主機配置？預算在45K，想要玩 POE、APEX等槍戰、LOL。',
    '預算約45K，需要一台主要用來跑Adobe系列軟體的電腦，特別是Photoshop和After Effects。',
    '我需要一台適合股市分析的電腦，預算大概30K，需要大量運算和多螢幕，有什麼好提議？',
    '想要組一台約65K的電腦，主要用於VR遊戲和一般辦公，有沒有推薦配置？',
    '請問4萬預算可以組出來能玩FPS、3A大作、多開模擬器遊戲的電腦嗎？',
    '預算25K，我想要一台電腦主要用於線上學習和偶爾寫程式，有建議配置嗎？',
    '請問75K左右的預算能組一台什麼樣的4K影片剪輯機？',
    '我要組一台電腦，預算約35K，主要用於玩《英雄聯盟》和做寫作業與論文研究，推薦配置是？',
    '預算大概55K，尋找一台適合做3D動畫和模型設計的電腦，有推薦嗎？',
    '我是遊戲開發學生，預算大約85K，需要一台能處理遊戲開發和渲染的高階機，有沒有好的配置？',
    '預算20K內，請問有適合辦公和輕度Photoshop使用的電腦配置嗎？',
    '想問下，有沒有大約50K的電腦配置，適合玩大型開放世界遊戲的？',
    '我需要一台電腦，預算大概45K，主要用於音樂製作和錄音，需要特別注意什麼配件？',
    '預算約60K，想要一台電腦主要用於科學計算和寫程式，有推薦配置嗎？',
    '請問我想組一台約70K的電腦，主要用於虛擬機運行和大數據處理，有什麼建議？',
    '預算大約30K，尋找一台適合初學者學習寫程式和偶爾玩遊戲的電腦，有推薦配置嗎？',
    '我要組裝一台預算約40K的電腦，主要用來看4K電影和日常使用，有建議配置嗎？',
    '想請問，85K預算能組一台什麼樣的專業級遊戲機？希望能玩最新遊戲。',
    '預算約50K，我想要一台電腦主要用於AI學習和研究，有沒有好的配置建議？',
    '請問一下，25K預算可以組一台適合高中生用的課業學習電腦嗎？主要是寫作業和上網。',
    '我是平面設計師，預算大概55K，需要一台適合設計工作的電腦，有推薦嗎？',
    '預算大約7萬，想要一台適合專業影像處理和影片剪輯的電腦，有什麼建議配置？',
    '我想組一台電腦，預算約60K，主要用於玩模擬飛行遊戲，有沒有推薦的配置？',
    '請問有沒有大約35K的電腦配置推薦，適合用於網頁開發和輕度遊戲？',
    '預算約80K，我需要一台適合進行生物資訊學研究的電腦，有什麼好的配置建議？',
    '我要組裝一台電腦，預算大概45K，主要用於網絡安全測試和寫程式，有推薦配置嗎？',
    '請問30K預算內，有沒有適合做基本影片剪輯和Photoshop的電腦配置？',
    '我想要一台約65K的電腦，主要用於遊戲開發和輕度3D建模，有什麼建議？',
    '預算大約75K，尋找一台適合高速股票交易和分析的電腦，需要考慮哪些配件？',
    '我需要一台電腦，預算約40K，主要用來做機器學習實驗和coding，有什麼好提議？',
    '請問55K預算能組一台什麼樣的電腦，適合做遊戲串流和錄製？',
    '想組一台預算約25K的電腦，主要用於日常使用和追劇，有推薦配置嗎？',
    '我是電腦小白，想組文書機偶爾玩玩LOL，有建議嗎？',
    '預算約20K，請問有適合老年人使用，簡單操作和看影片的電腦配置嗎？',
    '我想要一台電腦，預算大概50K，主要用於數位藝術創作和動畫製作，有建議配置嗎？',
    '請問65K預算內，適合玩高畫質遊戲和虛擬實境的電腦配置是？',
    '我需要一台約75K的電腦，主要用來做 deep learning 和大數據處理，有什麼配置建議？',
    '預算大約35K，尋找一台適合大學生學習寫 Python 和日常使用的電腦，有推薦嗎？',
    '我想組一台預算約45K的電腦，主要用於攝影後期製作和輕度遊戲，有推薦配置嗎？',
    '請問6萬預算能組出一台適合專業級音樂製作和錄音的電腦嗎？',
    '我要組一台電腦，預算大約55K，主要用於科學研究和數據分析，有什麼好建議？',
    '請問有沒有大約70000的電腦配置推薦，適合用於VR開發和高階遊戲？',
    '預算約30K，我需要一台適合初學者學習3D建模和動畫製作的電腦，有推薦配置嗎？',
    '我想組一台預算約35K的電腦，主要用於儲存照片跟檔案，有推薦嗎？',
    
]
for question in questions[:every_class_test_maximum]:
    
    data = {}
    data['qa_index'] = str(qa_index)
    qa_index += 1
    data['class'] = '4'
    data['question'] = question
    data['only_for_classification'] = only_for_classification
    test_set['data_list'].append(data)

顧問諮詢 資料建立

In [6]:
questions = [
    '30000文書機的電腦', 
    '5萬元的電競電腦', 
    '25000元以下的休閒用電腦', 
    '七萬元可以打3A遊戲的電腦', 
    '15000可以做上網、做報告的文書機'
    '預算五萬左右的遊戲用電腦', 
    '3萬元的電腦，可以防塵', 
    '6萬元的看影片電腦', 
    '2萬元以下簡單的文書電腦', 
    '3萬元的電腦，CPU指定使用Intel',
    '3萬內，主要是玩LOL和看Netflix的主機',
    '預算大概在50K，要能處理攝影後製和輕遊戲，如：軒轅劍7。',
    '碩士生研究用的電腦，預算約70K，需要跑深度學習模型',
    '預算大約20K，一台能寫報告和上網的簡易電腦，要求瀏覽器多開流暢。',
    '80K左右的遊戲主機，畫質要好。',
    '工作需要，一台能跑Solidworks和AutoCAD的電腦，預算大概60K',
    '遊戲主機配置，預算在45K，能玩 POE、APEX等槍戰、LOL。',
    '預算約45K，一台主要用來跑Adobe系列軟體的電腦，特別是Photoshop和After Effects。',
    '適合股市分析的電腦，預算大概30K，需要大量運算和多螢幕',
    '約65K的電腦，主要用於VR遊戲和一般辦公，有沒有推薦配置？',
    '4萬預算，可以玩FPS、3A大作、多開模擬器遊戲的電腦',
    '預算25K，主要用於線上學習和偶爾寫程式的電腦',
    '75K左右的4K影片剪輯機',
    '預算約35K，主要用於玩《英雄聯盟》和做寫作業與論文研究的電腦',
    '預算大概55K，適合做3D動畫和模型設計的電腦',
    '預算大約85K，需要一台能處理遊戲開發和渲染的高階機',
    '預算20K內，適合辦公和輕度Photoshop使用的電腦',
    '大約50K，適合玩大型開放世界遊戲的電腦',
    '預算約25K的電腦，主要用於日常使用和追劇',
    '想組文書機偶爾玩玩LOL',
    '預算約20K，適合老年人使用，簡單操作和看影片的電腦',
    ]

for question in questions[:every_class_test_maximum]:
    
    inputs =  {"question": question}
    answer = question_splitter.invoke(inputs)
    question_childs = re.findall(r'\d+\.\s*(.*)', answer)
    
    data = {}
    data['qa_index'] = str(qa_index)
    qa_index += 1
    data['class'] = '6'
    data['question'] = question
    data['question_childs'] = question_childs
    data['answer_childs'] = []
    data['only_for_classification'] = only_for_classification
    
    test_set['data_list'].append(data)

In [7]:
with open(test_dataset_path, 'w') as file:
    json.dump(test_set, file, ensure_ascii=False, indent=4)

print('Build test_dataset successfully')

Build test_dataset successfully


In [9]:
len(test_set['data_list'])

230